# Predizendo pontos marcadas por jogadores da NBA

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, f1_score, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

### Link para o dataset
https://www.kaggle.com/datasets/amirhosseinmirzaie/nba-players-stats2023-season

In [2]:
df = pd.read_csv("2023_nba_player_stats.csv")

df.head(3)

,PName,POS,Team,Age,GP,W,L,Min,PTS,FGM,...,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-
0,Jayson Tatum,SF,BOS,25,74,52,22,2732.2,2225,727,...,649,342,213,78,51,160,3691,31,1,470
1,Joel Embiid,C,PHI,29,66,43,23,2284.1,2183,728,...,670,274,226,66,112,205,3706,39,1,424
2,Luka Doncic,PG,DAL,24,66,33,33,2390.5,2138,719,...,569,529,236,90,33,166,3747,36,10,128


In [3]:
row, col = df.shape
print("This Dataset have", row, "rows and", col, "columns.")

This Dataset have 539 rows and 30 columns.


In [4]:
print("Number of duplicate data : ", df.duplicated().sum())

Number of duplicate data :  0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539 entries, 0 to 538
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PName   539 non-null    object 
 1   POS     534 non-null    object 
 2   Team    539 non-null    object 
 3   Age     539 non-null    int64  
 4   GP      539 non-null    int64  
 5   W       539 non-null    int64  
 6   L       539 non-null    int64  
 7   Min     539 non-null    float64
 8   PTS     539 non-null    int64  
 9   FGM     539 non-null    int64  
 10  FGA     539 non-null    int64  
 11  FG%     539 non-null    float64
 12  3PM     539 non-null    int64  
 13  3PA     539 non-null    int64  
 14  3P%     539 non-null    float64
 15  FTM     539 non-null    int64  
 16  FTA     539 non-null    int64  
 17  FT%     539 non-null    float64
 18  OREB    539 non-null    int64  
 19  DREB    539 non-null    int64  
 20  REB     539 non-null    int64  
 21  AST     539 non-null    int64  
 22  TO

In [6]:
df.describe(include=np.number)

,Age,GP,W,L,Min,PTS,FGM,FGA,FG%,3PM,...,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-
count,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,...,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000
mean,25.970315,48.040816,24.018553,24.022263,1103.617625,523.426716,191.576994,403.005566,46.325232,56.324675,...,198.254174,115.545455,61.300557,33.270872,21.241187,91.181818,1036.938776,4.011132,0.220779,0.000000
std,4.315513,24.650686,14.496366,13.445866,827.765114,498.084360,178.351286,369.595909,10.967271,60.916821,...,181.819962,129.578453,58.279185,28.336745,26.529238,66.206731,894.081896,8.770932,1.564432,148.223909
min,19.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,-642.000000
25%,23.000000,30.500000,12.000000,14.000000,329.000000,120.500000,45.500000,93.500000,41.650000,5.000000,...,50.500000,22.000000,14.500000,8.500000,5.000000,32.000000,254.000000,0.000000,0.000000,-70.000000
50%,25.000000,54.000000,25.000000,25.000000,970.200000,374.000000,138.000000,300.000000,45.500000,36.000000,...,159.000000,69.000000,44.000000,28.000000,13.000000,86.000000,810.000000,0.000000,0.000000,-7.000000
75%,29.000000,68.000000,36.000000,34.000000,1845.900000,769.500000,283.500000,598.500000,50.600000,92.000000,...,286.000000,162.500000,92.500000,51.000000,28.000000,140.000000,1646.000000,3.000000,0.000000,57.000000
max,42.000000,83.000000,57.000000,60.000000,2963.200000,2225.000000,728.000000,1559.000000,100.000000,301.000000,...,973.000000,741.000000,300.000000,128.000000,193.000000,279.000000,3842.000000,65.000000,29.000000,640.000000


In [7]:
df.describe(include="object")

,PName,POS,Team
count,539,534,539
unique,539,7,30
top,Jayson Tatum,SG,DAL
freq,1,96,21


In [8]:
df.isna().sum()

PName    0
POS      5
Team     0
Age      0
GP       0
W        0
L        0
Min      0
PTS      0
FGM      0
FGA      0
FG%      0
3PM      0
3PA      0
3P%      0
FTM      0
FTA      0
FT%      0
OREB     0
DREB     0
REB      0
AST      0
TOV      0
STL      0
BLK      0
PF       0
FP       0
DD2      0
TD3      0
+/-      0
dtype: int64

In [9]:
df["POS"].fillna("SG", inplace=True)

C:\Users\lgaqu\AppData\Local\Temp\ipykernel_4488\3416231543.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["POS"].fillna("SG", inplace=True)


In [10]:
px.histogram(
    df,
    x="POS",
    color_discrete_sequence=px.colors.qualitative.Vivid,
    title="Players position value counts",
    template="plotly_dark",
)

In [11]:
position_stats = df.groupby(["POS"]).agg({"PTS": "mean"}).reset_index()

fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=position_stats["POS"],
        y=position_stats["PTS"],
        marker=dict(
            color=[
                "orangered",
                "white",
                "grey",
                "orangered",
                "white",
                "grey",
                "orangered",
            ]
        ),
    )
)

fig.update_layout(
    title="Points per Position",
    xaxis_title="Position",
    yaxis_title="Average Total Points",
    template="plotly_dark",
)

fig.show()

In [12]:
fig_age_histogram = go.Figure()

fig_age_histogram.add_trace(go.Histogram(x=df["Age"], marker_color="orangered"))

fig_age_histogram.update_layout(
    title="Distribution of Player Ages",
    xaxis_title="Age",
    yaxis_title="Count",
    template="plotly_dark",
)

fig_age_histogram.show()

In [15]:
fig_total_points = px.scatter(
    df,
    x="Age",
    y="PTS",
    color="POS",
    title="Player Age vs Total Points",
    labels={"Age": "Age", "Total_Points": "Total Points"},
    template="plotly_dark",
)
fig_total_points.show()

fig_fg_percentage = px.scatter(
    df,
    x="Age",
    y="FG%",
    color="POS",
    title="Player Age vs Field Goal Percentage",
    labels={"Age": "Age", "Field_Goal_Percentage": "Field Goal Percentage"},
    template="plotly_dark",
)
fig_fg_percentage.show()

fig_assists = px.scatter(
    df,
    x="Age",
    y="AST",
    color="POS",
    title="Player Age vs Assists",
    labels={"Age": "Age", "Assists": "Assists"},
    template="plotly_dark",
)
fig_assists.show()